# 📥 Notebook 01: Ingest Crypto Data from CoinGecko API

This notebook ingests historical market data for selected cryptocurrencies using the public CoinGecko API. It prepares the data for further transformation and Delta Lake storage.


# 🧪 Step 1: Ingest Crypto Data from CoinGecko API

This notebook initializes the PySpark session and fetches historical price data for Bitcoin using the CoinGecko API.


In [7]:
from pyspark.sql import SparkSession

# Create Spark session
spark = SparkSession.builder \
    .appName("Crypto Ingestion") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")  # 👈 Hide warnings like SparkUI port binding, etc.

## 🔌 Step 2: Import API Utility Function

We will use a custom helper function from `src/api_utils.py` to retrieve 30 days of historical price data for Bitcoin.


In [8]:
import sys
sys.path.append("../src")

from api_utils import get_market_data

# Get Bitcoin data (returns Pandas DataFrame)
btc_df = get_market_data("bitcoin", "usd", 30)
btc_df.head()


,timestamp,price
0,2025-04-10 19:05:30.902,79529.676063
1,2025-04-10 20:09:28.195,79724.894660
2,2025-04-10 21:08:37.278,79878.623572
3,2025-04-10 22:05:26.534,79709.368773
4,2025-04-10 23:04:40.889,79715.508090


## 🔄 Step 3: Convert to Spark DataFrame

Now we convert the Pandas DataFrame to a Spark DataFrame for distributed processing.


In [9]:
# Convert to Spark DataFrame
btc_spark_df = spark.createDataFrame(btc_df)

# Display first 5 rows
btc_spark_df.show(5)

# Optional: Inspect schema
btc_spark_df.printSchema()


+--------------------+-----------------+
|           timestamp|            price|
+--------------------+-----------------+
|2025-04-10 19:05:...|79529.67606306932|
|2025-04-10 20:09:...|79724.89465979146|
|2025-04-10 21:08:...|79878.62357184727|
|2025-04-10 22:05:...|79709.36877328751|
|2025-04-10 23:04:...|  79715.508089801|
+--------------------+-----------------+
only showing top 5 rows

root
 |-- timestamp: timestamp (nullable = true)
 |-- price: double (nullable = true)

